<a href="https://colab.research.google.com/github/Jsooooooo/ML-DL/blob/master/QA_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

# 判断GPU还是CPU
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

/content/drive

In [ ]:
# 装载Google云盘到当前会话空间，那么云盘中的文件就在 /driver 目录下
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [ ]:
# 日志输出
def log(method,a,b):
  print("-------------------------")
  print(method,":")
  print(a,":",b)
  print("-------------------------")


# 1.格式化数据
加载数据集，是一个电影角色之间的对话
数据集是一些txt文件，首先要处理变为csv文件方便读取


In [ ]:
# 文件分割处理

# 将文件的每一行拆分为字段字典，key为ID，value为行
def loadLines(fileName, fields):
  lines = {}
  with open(fileName, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split(" +++$+++ ")

      # Extract fields
      lineObj = {}
      for i, field in enumerate(fields):
        lineObj[field] = values[i]
      lines[lineObj['lineID']] = lineObj
  return lines


# 将 `loadLines` 中的行字段分组为基于 *movie_conversations.txt* 的对话
def loadConversations(fileName, lines, fields):
  conversations = []
  with open(fileName, 'r', encoding='iso-8859-1') as f:
   for line in f:
     values = line.split(" +++$+++ ")

     # Extract fields
     convObj = {}
     for i, field in enumerate(fields):
       convObj[field] = values[i]

     # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
     lineIds = eval(convObj["utteranceIDs"])

     # Reassemble lines
     convObj["lines"] = []
     for lineId in lineIds:
       convObj["lines"].append(lines[lineId])
     conversations.append(convObj)
  return conversations


# 从对话中提取一对句子
def extractSentencePairs(conversations):
  qa_pairs = []
  for conversation in conversations:
   # Iterate over all the lines of the conversation
   for i in range(len(conversation["lines"]) - 1): # We ignore the last line(no answer for it)
     inputLine = conversation["lines"][i]["text"].strip()
     targetLine = conversation["lines"][i+1]["text"].strip()
     # Filter wrong samples (if one of the lists is empty)
     if inputLine and targetLine:
       qa_pairs.append([inputLine, targetLine])
  return qa_pairs

In [ ]:
# 将处理好的写入新txt文件，再写入csv

# 源文件路径
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("/drive/My Drive/Dataset", corpus_name)

# 定义新文件的路径
datafile = os.path.join(corpus, "formatted_movie_lines.txt")
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# 初始化行dict，对话列表和字段ID
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# 加载行和进程对话
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),lines, MOVIE_CONVERSATIONS_FIELDS)

# 写入新的csv文件
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
  writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
  for pair in extractSentencePairs(conversations):
    writer.writerow(pair)
# 打印一个样本的行
print("\nSample lines from file:")

def printLines(file, n=10):
  with open(file, 'rb') as datafile:
    lines = datafile.readlines()
  for line in lines[:n]:
    print(line)
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't dat

# 2.加载和清洗数据
1.   加载
  1.   首先创建词汇表（Vocabulary），并将这些句子转换为句子对
  2.   创建一个Voc类，存储单词到索引的映射，索引到单词反映射，每个句子单词计数和总单词量
  3.   Voc类方法：添加单个单词、添加句子中所有单词、清洗计数低于阈值的单词
2.   清洗
  1.   将unicode转为ASCII码
  2.   将所有字母转为小写
  3.   清洗除基本标点之外的字符
  4.   过滤超出最大长度的句子









In [2]:
# 默认词向量
PAD_token = 0 # 用来填充短句子
SOS_token = 1 # Start-of-sentence 标记
EOS_token = 2 # End-of-sentence 标记

class Voc:
  def __init__(self, name):
    # 单词表名字
    self.name = name
    # 是否已修建标识
    self.trimmed = False
    # 单词到索引的映射字典 {单词:当前位置索引}
    self.word2index = {}
    # 某个单词的数目字典  {单词:出现次数}
    self.word2count = {}
    # 索引到单词的映射字典 {当前位置索引:单词}
    self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
    # 单词表的大小（当前位置）
    self.num_words = 3 # Count SOS, EOS, PAD
  
  # 添加句子中的所有单词，先分割
  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)
  
  # 添加单个单词到单词表
  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.num_words
      self.word2count[word] = 1
      self.index2word[self.num_words] = word
      self.num_words += 1
    else:
      self.word2count[word] += 1
  
  # 删除低于特定计数阈值的单词
  def trim(self, min_count):
    # 排除已经修剪过的
    if self.trimmed:
      return
    self.trimmed = True
    
    keep_words = []
    for k, v in self.word2count.items():
      if v >= min_count:
        keep_words.append(k)
      print('keep_words {} / {} = {:.4f}'.format(
      len(keep_words), len(self.word2index), len(keep_words) /
      len(self.word2index)
      ))
    
    # 重初始化字典
    self.word2index = {}
    self.word2count = {}
    self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
    self.num_words = 3 # Count default tokens
    
    # 修剪完，清空所有映射数组，重新讲这些修剪后的单词加入单词表
    for word in keep_words:
      self.addWord(word)

In [4]:
def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
  if unicodedata.category(c) != 'Mn'
  )

ii = Voc("voc")
ii.addSentence(unicodeToAscii("I want new balance"))
ii.addSentence(unicodeToAscii("no new balance doesn't fit me"))
print (ii.word2index)
print (ii.word2count)
print (ii.index2word)
print (ii.num_words)

{'I': 3, 'want': 4, 'new': 5, 'balance': 6, 'no': 7, "doesn't": 8, 'fit': 9, 'me': 10}
{'I': 1, 'want': 1, 'new': 2, 'balance': 2, 'no': 1, "doesn't": 1, 'fit': 1, 'me': 1}
{0: 'PAD', 1: 'SOS', 2: 'EOS', 3: 'I', 4: 'want', 5: 'new', 6: 'balance', 7: 'no', 8: "doesn't", 9: 'fit', 10: 'me'}
11


In [ ]:
MAX_LENGTH = 10 # Maximum sentence length to consider

# 将Unicode字符串转换为纯ASCII，多亏了
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
  if unicodedata.category(c) != 'Mn'
  )


# 初始化Voc对象，并把格式化pairs对话存放到list中
def readVocs(datafile, corpus_name):
  print("Reading lines...")
  # 读取文件，并按行分割
  lines = open(datafile, encoding='utf-8').read().strip().split('\n')

  # 每行都是以 \t 分割句子对，将每个句子中的单词加入单词表
  pairs = [[unicodeToAscii(s) for s in l.split('\t')] for l in lines]
  # 创建一个单词表对象，命名为数据集的名字
  voc = Voc(corpus_name)
  return voc, pairs


# 如果对 'p' 中（一行数据，即一个句子对）的两个句子都低于 MAX_LENGTH 阈值，则返回True
def filterPair(p):
  # Input sequences need to preserve the last word for EOS token
  return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH


# 过滤满足条件的 pairs 对话（对话即一行，两个句子）
def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]


# 使用上面定义的函数，返回一个填充的voc对象和对列表
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
  print("Start preparing training data ...")
  voc, pairs = readVocs(datafile, corpus_name)
  print("读取 {!s} 个句子对".format(len(pairs)))
  pairs = filterPairs(pairs)
  print("清洗后剩余 {!s} 个句子对".format(len(pairs)))
  print("Counting words...")
  for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
  print("Counted words:", voc.num_words)
  return voc, pairs


# 加载voc和句子对（voc、pairs）
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# 打印一些对进行验证
print("\npairs:")
for pair in pairs[:10]:
  print(pair)

Start preparing training data ...
Reading lines...
读取 221282 个句子对
清洗后剩余 88265 个句子对
Counting words...
Counted words: 61758

pairs:
['Gosh, if only we could find Kat a boyfriend...', 'Let me see what I can do.']
["C'esc ma tete. This is my head", "Right.  See?  You're ready for the quiz."]
["That's because it's such a nice one.", 'Forget French.']
['There.', 'Where?']
['You have my word.  As a gentleman', "You're sweet."]
['Hi.', 'Looks like things worked out tonight, huh?']
['You know Chastity?', 'I believe we share an art instructor']
['Have fun tonight?', 'Tons']
['Well, no...', "Then that's all you had to say."]
["Then that's all you had to say.", 'But']


# 3.词嵌入-为模型提供矩阵数据
1.   将数据转化为torch的tensor形式
2.   seq2seq模型中的batch_size设为1，说明是把
3.   使用GPU并行运算，需用mini_batch
4.   但是mini_batch每个句子长短不同，所以需将短句子进行补0填充
5.   

In [5]:
# 返回一个句子中单词所对应的索引组成的列表
def indexesFromSentence(voc, sentence):
  # log("indexesFromSentence","sentence",sentence)
  return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

print (indexesFromSentence(ii,"I no want balance"))

[3, 7, 4, 6, 2]


In [ ]:
# 返回一个句子中单词所对应的索引组成的列表
def indexesFromSentence(voc, sentence):
  log("indexesFromSentence","sentence",sentence)
  return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


# zip_longest 对数据进行转置，并补0操作
def zeroPadding(l, fillvalue=PAD_token):
  # zip_longest:将元素行转置成一列，并在末尾补0
  # 单个星号接受多个非字典值形成一个列表
  log("zeroPadding","l",l)
  log("zeroPadding","l padding",list(itertools.zip_longest(*l, fillvalue=fillvalue)))
  return list(itertools.zip_longest(*l, fillvalue=fillvalue))


# 用一个矩阵m 记录PAD_token填充的的位置为0，其他的为1
def binaryMatrix(l, value=PAD_token):
  m = []
  for i, seq in enumerate(l):
    m.append([])
    for token in seq:
      if token == PAD_token:
        m[i].append(0)
      else:
        m[i].append(1)
  return m


# 返回填充前句子对的前一句（即问题句）的长度和填充后的输入序列张量
def inputVar(l, voc):
  indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
  lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  padVar = torch.LongTensor(padList)
  return padVar, lengths


# 返回填充前句子对的后一句（即回答句）最长的一个长度和填充后的输入序列张量,和填充后的标记矩阵mask
def outputVar(l, voc):
  indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
  max_target_len = max([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  mask = binaryMatrix(padList)
  mask = torch.ByteTensor(mask)
  padVar = torch.LongTensor(padList)
  return padVar, mask, max_target_len


# 返回给定batch对的所有项目
def batch2TrainData(voc, pair_batch):
  pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
  input_batch, output_batch = [], []
  for pair in pair_batch:
    input_batch.append(pair[0])
    output_batch.append(pair[1])
  inp, lengths = inputVar(input_batch, voc)
  output, mask, max_target_len = outputVar(output_batch, voc)
  return inp, lengths, output, mask, max_target_len


# 验证例子
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches
print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

-------------------------
indexesFromSentence :
sentence : I am the Lord's voice in this town.
-------------------------
-------------------------
indexesFromSentence :
sentence : And still Mrs. Harold Ryan?
-------------------------
-------------------------
indexesFromSentence :
sentence : I like you, Dil --
-------------------------
-------------------------
indexesFromSentence :
sentence : It's 350 pages long.
-------------------------
-------------------------
indexesFromSentence :
sentence : Where is your mother?
-------------------------
-------------------------
zeroPadding :
l : [[16, 758, 32, 8209, 5615, 140, 81, 6183, 2], [228, 676, 1439, 10573, 14799, 2], [16, 52, 906, 37950, 216, 2], [146, 25496, 25497, 7570, 2], [122, 23, 114, 3718, 2]]
-------------------------
-------------------------
zeroPadding :
l padding : [(16, 228, 16, 146, 122), (758, 676, 52, 25496, 23), (32, 1439, 906, 25497, 114), (8209, 10573, 37950, 7570, 3718), (5615, 14799, 216, 2, 2), (140, 2, 2, 0, 0), 


zip_longest函数用法
```
res = zip_longest('abc', '12')
for x in res:
    print(x)
```

结果
```
('a', '1')
('b', '2')
('c', None)
```




# 4.定义Seq2Seq模型
1.   首先将上述单词索引经过处理得到的tensor数据进行词嵌入
2.   为RNN模块打包填充batch序列。
3.   通过GRU进行前向传播。
4.   反填充。
5.   对双向GRU输出求和。
6.   返回输出和最终隐藏状态。



In [ ]:
# 编码器GRU

class EncoderRNN(nn.Module):
  def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
    super(EncoderRNN, self).__init__()
    self.n_layers = n_layers
    self.hidden_size = hidden_size
    self.embedding = embedding

    # 初始化GRU; input_size和hidden_size参数都设置为'hidden_size'
    # 因为我们的输入大小是一个嵌入了多个特征的单词== hidden_size
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers,dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
 
 

  def forward(self, input_seq, input_lengths, hidden=None):
    # 将单词索引转换为词向量
    embedded = self.embedding(input_seq)
    # 为RNN模块打包填充batch序列
    packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
    # 正向通过GRU
    outputs, hidden = self.gru(packed, hidden)
    # 打开填充
    outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
    # 总和双向GRU输出
    outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
    # 返回输出和最终隐藏状态
    return outputs, hidden

In [ ]:
# 注意力子模块
# Luong的attention layer
class Attn(torch.nn.Module):
  def __init__(self, method, hidden_size):
    super(Attn, self).__init__()
    
    self.method = method
    

    # 判断注意力计算方式
    if self.method not in ['dot', 'general', 'concat']:
      raise ValueError(self.method, "is not an appropriate attention method.")
    
    self.hidden_size = hidden_size

    # 初始化attention，使用一个小神经网络自主学习
    if self.method == 'general':
      self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
    elif self.method == 'concat':
      self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
      self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))


  # 三种计算注意力能量的方式
  def dot_score(self, hidden, encoder_output):
    return torch.sum(hidden * encoder_output, dim=2)

  def general_score(self, hidden, encoder_output):
    energy = self.attn(encoder_output)
    return torch.sum(hidden * energy, dim=2)

  def concat_score(self, hidden, encoder_output):
    energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
    return torch.sum(self.v * energy, dim=2)


  def forward(self, hidden, encoder_outputs):
    # 根据给定的方法计算注意力（能量） 
    if self.method == 'general':
      attn_energies = self.general_score(hidden, encoder_outputs)
    elif self.method == 'concat':
      attn_energies = self.concat_score(hidden, encoder_outputs)
    elif self.method == 'dot':
      attn_energies = self.dot_score(hidden, encoder_outputs)
    
    # Transpose max_length and batch_size dimensions
    attn_energies = attn_energies.t()
    # Return the softmax normalized probability scores (with added dimension)
    return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
# 解码器GRU
class LuongAttnDecoderRNN(nn.Module):
  def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
    super(LuongAttnDecoderRNN, self).__init__()

    self.attn_model = attn_model
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    self.dropout = dropout

    # 定义层
    self.embedding = embedding
    self.embedding_dropout = nn.Dropout(dropout)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
    self.concat = nn.Linear(hidden_size * 2, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.attn = Attn(attn_model, hidden_size)
  
  
  def forward(self, input_step, last_hidden, encoder_outputs):
    # 注意：我们一次运行这一步（单词）
    # 获取当前输入字的嵌入
    embedded = self.embedding(input_step)
    embedded = self.embedding_dropout(embedded)
    
    # 通过单向GRU转发
    rnn_output, hidden = self.gru(embedded, last_hidden)
    
    # 从当前GRU输出计算注意力
    attn_weights = self.attn(rnn_output, encoder_outputs)
    
    # 将注意力权重乘以编码器输出以获得新的“加权和”上下文向量
    context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
    
    # 使用Luong的公式五连接加权上下文向量和GRU输出
    rnn_output = rnn_output.squeeze(0)
    context = context.squeeze(1)
    concat_input = torch.cat((rnn_output, context), 1)
    concat_output = torch.tanh(self.concat(concat_input))
    
    # 使用Luong的公式6预测下一个单词
    output = self.out(concat_output)
    output = F.softmax(output, dim=1)
    
    # 返回输出和在最终隐藏状态
    return output, hidden

# 5.定义训练步骤
1.   定义损失函数
2.   定义单次训练步骤
3.   开始迭代训练
4.   每次迭代保存参数





In [ ]:
# 计算平均负对数似然作为损失函数
# 但是我们需要排除填充的0对应的tensor中的值，因为是0填充的

def maskNLLLoss(inp, target, mask):
  nTotal = mask.sum()
  crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
  loss = crossEntropy.masked_select(mask).mean()
  loss = loss.to(device)
  return loss, nTotal.item()

In [ ]:
# 单次训练

def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):
  # 零化梯度
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  
  # 设置设备选项
  input_variable = input_variable.to(device)
  lengths = lengths.to(device)
  target_variable = target_variable.to(device)
  mask = mask.to(device)
  
  # 初始化变量
  loss = 0
  print_losses = []
  n_totals = 0
  
  # 正向传递编码器
  encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
  
  # 创建初始解码器输入（从每个句子的SOS令牌开始）
  decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
  decoder_input = decoder_input.to(device)
  
  # 将初始解码器隐藏状态设置为编码器的最终隐藏状态
  decoder_hidden = encoder_hidden[:decoder.n_layers]
  
  # 确定我们是否此次迭代使用`teacher forcing`
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
  
  # 通过解码器一次一步地转发一批序列
  if use_teacher_forcing:
    for t in range(max_target_len):
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
  
      # Teacher forcing: 下一个输入是当前的目标
      decoder_input = target_variable[t].view(1, -1)

      # 计算并累计损失
      mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t],mask[t])
      loss += mask_loss
      print_losses.append(mask_loss.item() * nTotal)
      n_totals += nTotal
  else:
    for t in range(max_target_len):
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
  
      # No teacher forcing: 下一个输入是解码器自己的当前输出
      _, topi = decoder_output.topk(1)
      decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
      decoder_input = decoder_input.to(device)

      # 计算并累计损失
      mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
      loss += mask_loss
      print_losses.append(mask_loss.item() * nTotal)
      n_totals += nTotal
  
  # 执行反向传播
  loss.backward()
  
  # 剪辑梯度：梯度被修改到位
  _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
  _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
  
  # 调整模型权重
  encoder_optimizer.step()
  decoder_optimizer.step()
  return sum(print_losses) / n_totals 

In [ ]:
# 迭代训练
# 并为每次迭代保存模型参数

def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, 
               decoder_optimizer, embedding, encoder_n_layers, 
               decoder_n_layers, save_dir, n_iteration, batch_size, 
               print_every, save_every, clip, corpus_name, loadFilename):
  
  # 为每次迭代加载batches（提前将所有轮次的batch提取出来放在列表）
  training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)]) for _ in range(n_iteration)]
  
  # 初始化
  print('Initializing ...')
  start_iteration = 1
  print_loss = 0
  if loadFilename:
    start_iteration = checkpoint['iteration'] + 1
  

  # 开始循环训练
  print("Training...")
  for iteration in range(start_iteration, n_iteration + 1):
    training_batch = training_batches[iteration - 1]
    
    # 从batch中提取字段
    input_variable, lengths, target_variable, mask, max_target_len = training_batch
    
    # 使用train函数对batch进行训练
    loss = train(input_variable, lengths, target_variable, mask, 
                 max_target_len, encoder,decoder, embedding,
                 encoder_optimizer, decoder_optimizer, 
                 batch_size, clip)
    print_loss += loss
    
    # 打印进度
    if iteration % print_every == 0:
      print_loss_avg = print_loss / print_every
      print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}"
       .format(iteration, iteration / n_iteration * 100, print_loss_avg))
      print_loss = 0
    
    # 每save_every轮就保存一次checkpoint
    if (iteration % save_every == 0):
      directory = os.path.join("/drive/My Drive/Model/cornell movie-dialogs corpus", model_name, corpus_name, '{}-{}_{}'
        .format(encoder_n_layers, decoder_n_layers, hidden_size))
      # 查看是否存在文件目录
      if not os.path.exists(directory):
        os.makedirs(directory)

      # 保存
      torch.save({
      'iteration': iteration,
      'en': encoder.state_dict(),
      'de': decoder.state_dict(),
      'en_opt': encoder_optimizer.state_dict(),
      'de_opt': decoder_optimizer.state_dict(),
      'loss': loss,
      'voc_dict': voc.__dict__,
      'embedding': embedding.state_dict()
      }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [ ]:
# 贪婪解码：搜索解码器输出的decoder_output中的softmax值最大的单词

class GreedySearchDecoder(nn.Module):
  def __init__(self, encoder, decoder):
    super(GreedySearchDecoder, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
  
  def forward(self, input_seq, input_length, max_length):
    # 通过编码器模型转发输入
    encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
    
    # 准备编码器的最终隐藏层作为解码器的第一个隐藏输入
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    
    # 使用SOS_token初始化解码器输入
    decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
    
    # 初始化张量以将解码后的单词附加到
    all_tokens = torch.zeros([0], device=device, dtype=torch.long)
    all_scores = torch.zeros([0], device=device)
    
    # 一次迭代地解码一个词tokens
    for _ in range(max_length):
      # 正向通过解码器
      decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
      # 获得最可能的单词标记及其softmax分数
      decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
      # 记录token和分数
      all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
      all_scores = torch.cat((all_scores, decoder_scores), dim=0)
      # 准备当前令牌作为下一个解码器输入（添加维度）
      decoder_input = torch.unsqueeze(decoder_input, 0)
    # 返回收集到的词tokens和分数
    return all_tokens, all_scores


#  6.定义聊天输入模块，并初始化输入

In [ ]:
# 跟机器人聊天时输入的文本，进行评估初始化

def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
  ### 格式化输入句子作为batch
  # words -> indexes
  indexes_batch = [indexesFromSentence(voc, sentence)]
  
  # 创建lengths张量
  lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
  
  # 转置batch的维度以匹配模型的期望
  input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
  
  # 使用合适的设备
  input_batch = input_batch.to(device)
  lengths = lengths.to(device)
  
  # 用searcher解码句子
  tokens, scores = searcher(input_batch, lengths, max_length)
  
  # indexes -> words
  decoded_words = [voc.index2word[token.item()] for token in tokens]
  return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
  input_sentence = ''
  while(1):
    try:
      # 获取输入句子
      input_sentence = input('> ')
      
      # 检查是否退出
      if input_sentence == 'q' or input_sentence == 'quit': 
        break
      
      # 规范化句子
      input_sentence = unicodeToAscii(input_sentence)
      
      # 评估句子
      output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
      
      # 格式化和打印回复句
      output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
      print('Bot:', ' '.join(output_words))
    except KeyError:
      print("Error: Encountered unknown word.")


# 7.运行模型
1.   初始化各个编码器解码器
2.   配置从头开始训练还是从checkpoint开始



In [ ]:
# 配置模型
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# 设置检查点以加载; 如果从头开始，则设置为None
loadFilename = "/drive/My Drive/Model/cornell movie-dialogs corpus/cb_model/cornell movie-dialogs corpus/2-2_500/4000_checkpoint.tar"
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
# '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
# '{}_checkpoint.tar'.format(checkpoint_iter))


# 如果提供了loadFilename（checkpoint地址），则加载模型
if loadFilename:
  # 如果在同一台机器上加载，则对模型进行训练
  checkpoint = torch.load(loadFilename)
  # If loading a model trained on GPU to CPU
  #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
  encoder_sd = checkpoint['en']
  decoder_sd = checkpoint['de']
  encoder_optimizer_sd = checkpoint['en_opt']
  decoder_optimizer_sd = checkpoint['de_opt']
  embedding_sd = checkpoint['embedding']
  voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# 初始化词向量
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
  embedding.load_state_dict(embedding_sd)

# 初始化编码器 & 解码器模型
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, 
                              decoder_n_layers, dropout)
if loadFilename:
  encoder.load_state_dict(encoder_sd)
  decoder.load_state_dict(decoder_sd)

# 使用合适的设备
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
# 忽略warning
import warnings
 
warnings.filterwarnings('ignore')

In [ ]:
# 执行云训练

# 配置训练/优化
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# 确保dropout layers在训练模型中
encoder.train()
decoder.train()

# 初始化优化器
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate *
decoder_learning_ratio)
if loadFilename:
  encoder_optimizer.load_state_dict(encoder_optimizer_sd)
  decoder_optimizer.load_state_dict(decoder_optimizer_sd)


# 运行训练迭代
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, 
           decoder_optimizer,embedding, encoder_n_layers, decoder_n_layers, 
           save_dir, n_iteration, batch_size,print_every, save_every, 
           clip, corpus_name, loadFilename)



Building optimizers ...
Starting Training!
Initializing ...
Training...


# 8.和模型聊天

In [ ]:
# 将dropout layers设置为eval模式
encoder.eval()
decoder.eval()

# 初始化探索模块
searcher = GreedySearchDecoder(encoder, decoder)

# 开始聊天（取消注释并运行以下行开始）
evaluateInput(encoder, decoder, searcher, voc)

> hello
Bot: I don't know. a lot.
> where?
Bot: I don't know. to be here --
> are you ok?
Bot: I'm okay. I'm a cab.
> who are you
Bot: I don't know. and Andy."""
> what's wrong
Bot: What's the matter? to the picture?
> are you kidding me
Bot: I don't know.
> do you know red velvet?
Error: Encountered unknown word.
> do you know James?
Bot: I know. the first time. the bricks.
> Obama
Error: Encountered unknown word.
> what for breakfast?
Bot: You know what I mean.
> ok
Bot: You know what I mean, you're going to do?
> shut up
Bot: I didn't have a bad thing to be here.
> how are you?
Bot: I'm okay. a little far. I'm okay.
> let's go
Bot: I don't know. a little lady.
> play basketball
Bot: I don't know what you want to know.
> do you understand?
Bot: I don't know.
> don't say anything
Bot: I don't know what I want.
> son of bitch
Bot: I don't know where about the corner.
> fuck
Bot: You know what I mean, I'm afraid you were married.
> are you married?
Bot: I don't know.
> quit


In [ ]:
b_list = [1,3,2,4]
print([b_list,b_list])
print(*[b_list,b_list])
print(*zip(zip(*[b_list,b_list])))

[[1, 3, 2, 4], [1, 3, 2, 4]]
[1, 3, 2, 4] [1, 3, 2, 4]
((1, 1),) ((3, 3),) ((2, 2),) ((4, 4),)
